# Part A: Property Price Prediction

Predict the **median house value** using district-level features. This notebook covers EDA, preprocessing, **Simple Linear Regression** and **Multiple Linear Regression**, plus evaluation with **MSE**, **RMSE**, and **R²**.


# Setup

This notebook installs/imports commonly used libraries.  
If some libraries are missing in your environment, run the install cell below.


In [ ]:

# If needed, uncomment to install dependencies in your environment
# %pip install pandas numpy scikit-learn matplotlib seaborn joblib openpyxl


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Regression
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

# Classification
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, RocCurveDisplay)
import joblib

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)


In [ ]:

def read_data_flex(path_or_url: str):
    """Load CSV/Excel from a local path or URL (Google Sheets/Drive direct link supported).
    If it's a Google Sheet viewing URL, convert it to CSV export automatically.
    """
    p = str(path_or_url).strip()
    # Try to convert Google Sheets view link to export CSV
    if "docs.google.com/spreadsheets" in p and "/edit" in p and "export?format=csv" not in p:
        # Convert to CSV export
        key = p.split("/d/")[1].split("/")[0]
        p = f"https://docs.google.com/spreadsheets/d/{key}/export?format=csv"
    try:
        if p.lower().endswith((".xlsx", ".xls")):
            df = pd.read_excel(p)
        else:
            df = pd.read_csv(p)
    except Exception as e:
        raise RuntimeError(f"Failed to load data from '{path_or_url}'. Error: {e}")
    return df

def quick_info(df: pd.DataFrame, name: str = "Data"):
    print(f"\n{name} shape: {df.shape}\n")
    display(df.head())
    display(df.describe(include='all').transpose())
    print("\nMissing values per column:\n")
    display(df.isna().sum().to_frame('missing'))
    print("\nDtypes:\n")
    display(df.dtypes.to_frame('dtype'))

def numeric_categorical_columns(df: pd.DataFrame, exclude_target: str = None):
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = df.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
    if exclude_target and exclude_target in num_cols:
        num_cols.remove(exclude_target)
    if exclude_target and exclude_target in cat_cols:
        cat_cols.remove(exclude_target)
    return num_cols, cat_cols


## 1) Load Data

- Provide either a local CSV/Excel path or a Google Sheet link.
- The assignment mentions California housing data and a categorical feature like `ocean_proximity`.
- Update `DATA_PATH_OR_URL` below to your actual file or the provided link.

In [ ]:

# === User input: set your data path or Google Sheet link ===
# Example (Google Sheet from assignment doc): paste the link below
DATA_PATH_OR_URL = "https://docs.google.com/spreadsheets/d/1B9RTjgPSEWyQzDr-CQJmCYRriaBIaGq2WEkfp2TLTIA/edit?usp=sharing"

df = read_data_flex(DATA_PATH_OR_URL)
quick_info(df, "Raw Data")


## 2) Basic EDA & Visualizations

In [ ]:

# Histograms for numeric columns
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if num_cols:
    df[num_cols].hist(bins=30, figsize=(14, 10))
    plt.tight_layout()
    plt.show()

# Correlation heatmap
if len(num_cols) > 1:
    plt.figure(figsize=(10, 8))
    sns.heatmap(df[num_cols].corr(), annot=False)
    plt.title("Correlation Heatmap (Numeric Features)")
    plt.show()


## 3) Train/Test Split & Preprocessing

- Target column expected: **`median_house_value`** (change if your file uses a different name)
- Simple imputation for missing values
- One-hot encoding for categoricals
- Standard scaling for numerics (for some models)

In [ ]:

TARGET = "median_house_value"  # <-- change if needed

assert TARGET in df.columns, f"Target column '{TARGET}' not found in data. Please update TARGET."

X = df.drop(columns=[TARGET])
y = df[TARGET]

num_cols, cat_cols = numeric_categorical_columns(X)

numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(transformers=[
    ("num", numeric_pipe, num_cols),
    ("cat", categorical_pipe, cat_cols)
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")


## 4) Simple Linear Regression (Single Feature)

We'll use `median_income` as a common strong predictor. If your dataset has a different income-related column, update below.

In [ ]:

single_feature = "median_income"  # change if needed
assert single_feature in X.columns, f"'{single_feature}' not in columns. Please set to a valid numeric column."

X_train_sf = X_train[[single_feature]]
X_test_sf = X_test[[single_feature]]

slr = LinearRegression()
slr.fit(X_train_sf, y_train)
pred_sf = slr.predict(X_test_sf)

mse_sf = mean_squared_error(y_test, pred_sf)
rmse_sf = np.sqrt(mse_sf)
r2_sf = r2_score(y_test, pred_sf)

print("Simple Linear Regression (feature = {}):".format(single_feature))
print(f"MSE: {mse_sf:.2f}, RMSE: {rmse_sf:.2f}, R^2: {r2_sf:.4f}")


## 5) Multiple Linear Regression & Regularized Models

We compare **Linear Regression**, **Ridge**, and **Lasso** using a pipeline.

In [ ]:

models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(max_iter=10000)
}

param_grid = {
    "Ridge": {"model__alpha": [0.1, 1.0, 10.0]},
    "Lasso": {"model__alpha": [0.001, 0.01, 0.1, 1.0]}
}

results = []
best_pipelines = {}

for name, model in models.items():
    pipe = Pipeline(steps=[("preprocess", preprocess),
                          ("model", model)])
    if name in param_grid:
        grid = GridSearchCV(pipe, param_grid=param_grid[name], cv=5, scoring="r2", n_jobs=-1)
        grid.fit(X_train, y_train)
        best_pipe = grid.best_estimator_
        best_pipelines[name] = best_pipe
        y_pred = best_pipe.predict(X_test)
        used_model = best_pipe.named_steps["model"]
        best_params = grid.best_params_
    else:
        pipe.fit(X_train, y_train)
        best_pipelines[name] = pipe
        y_pred = pipe.predict(X_test)
        used_model = model
        best_params = {}
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    results.append((name, mse, rmse, r2, best_params))

res_df = pd.DataFrame(results, columns=["Model", "MSE", "RMSE", "R2", "BestParams"])
display(res_df.sort_values("R2", ascending=False))

# Save the best model by R2
best_name = res_df.sort_values("R2", ascending=False).iloc[0]["Model"]
joblib.dump(best_pipelines[best_name], "partA_best_model.joblib")
print(f"Saved best Part A model as 'partA_best_model.joblib' ({best_name}).")


## 6) Conclusions

- Compare metrics and choose the model with highest R² and reasonable RMSE.
- Interpretability tip: Linear/Ridge/Lasso offer coefficients you can inspect.